# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f16cbba3490>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'H7_Vaccination policy', 'H7_Flag',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndex',
       'StringencyLegacyIndexForDispla

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92693,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92694,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92695,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
92696,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=42)

In [16]:
!pip install lightgbm

In [17]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
# model = Lasso(alpha=0.1,
#               precompute=True,
#               max_iter=10000,
#               positive=True,
#               selection='random')

# model = Lasso()
# # Fit model
# model.fit(X_train, y_train)

In [35]:
# change from Lasso to lightgbm

In [36]:
import random

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    
seed_everything(42) 

import lightgbm as lgb
NUM_ESTIMATORS = 10000
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'mae',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.05,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': NUM_ESTIMATORS,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# set negative values to zeros
y_train = np.maximum(y_train, 0)
train_data = lgb.Dataset(X_train,  label=y_train)
model = lgb.train(lgb_params,train_data,valid_sets = [train_data],verbose_eval = NUM_ESTIMATORS//20)

[500]	training's l1: 116.695
[1000]	training's l1: 106.821
[1500]	training's l1: 101.451
[2000]	training's l1: 97.5733
[2500]	training's l1: 94.5429
[3000]	training's l1: 91.9153
[3500]	training's l1: 89.5364
[4000]	training's l1: 87.0959
[4500]	training's l1: 85.2548
[5000]	training's l1: 83.5195
[5500]	training's l1: 82.0762
[6000]	training's l1: 80.2533
[6500]	training's l1: 78.7294
[7000]	training's l1: 77.341
[7500]	training's l1: 75.9138
[8000]	training's l1: 74.5701
[8500]	training's l1: 73.4322
[9000]	training's l1: 72.4657
[9500]	training's l1: 71.2562
[10000]	training's l1: 70.2531


In [37]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 70.25309058587398
Test MAE: 116.10916097364483


In [38]:
# Lasso:
# Train MAE: 125.79058821618477
# Test MAE: 126.2785299934449

In [39]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
# for (col, coeff) in zip(x_col_names, list(model.coef_)):
#     if coeff != 0.:
#         print(col, coeff)
# print('Intercept', model.intercept_)

In [40]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [41]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [42]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 11.290425620760875
2020-08-02: 10.039194205035018
2020-08-03: 11.425396219616268
2020-08-04: 11.526868044645925
2020-08-05: 8.034062856753383
2020-08-06: 6.431140855041713
2020-08-07: 6.736066034225573
2020-08-08: 10.325793490547667
2020-08-09: 11.96626145102764
2020-08-10: 11.849329190262857
2020-08-11: 10.374685300155791
2020-08-12: 7.993197856354013
2020-08-13: 6.880708960766592
2020-08-14: 6.1352551059722735
2020-08-15: 9.268891994530339
2020-08-16: 9.974877258940714
2020-08-17: 8.344708257281528
2020-08-18: 9.419041541459475
2020-08-19: 9.061489094901795
2020-08-20: 6.363561336165582
2020-08-21: 6.1854112816203966
2020-08-22: 8.949982940247983
2020-08-23: 9.738967238066758
2020-08-24: 8.074845987496918
2020-08-25: 8.330441489704102
2020-08-26: 7.301027393929616
2020-08-27: 7.503636365583012
2020-08-28: 5.726029743555302
2020-08-29: 8.240017506885856
2020-08-30: 8.473863594261495
2020-08-31: 7.444958094791111

Predicting for Afghanistan__nan
2

2020-08-18: 672.0360733685918
2020-08-19: 640.3213064606407
2020-08-20: 579.7937234669079
2020-08-21: 425.73079908484647
2020-08-22: 514.1875903510564
2020-08-23: 479.48220226355846
2020-08-24: 341.4840642498888
2020-08-25: 290.69663738233794
2020-08-26: 399.3287361836964
2020-08-27: 309.9580592671162
2020-08-28: 216.8862075431163
2020-08-29: 176.28613450563694
2020-08-30: 275.78789253672
2020-08-31: 165.3249548816385

Predicting for Burundi__nan
2020-08-01: 4.107614375447628
2020-08-02: 1.5429606003266076
2020-08-03: 1.2302855951153444
2020-08-04: 1.6811826800581267
2020-08-05: 2.667128661751726
2020-08-06: 2.4773955852353313
2020-08-07: 2.5552690466539514
2020-08-08: 2.934190336524981
2020-08-09: 3.043987846134058
2020-08-10: 3.9809475075479153
2020-08-11: 4.294979820222463
2020-08-12: 5.679298279375914
2020-08-13: 5.282051976063204
2020-08-14: 9.36458350904861
2020-08-15: 17.656365654938828
2020-08-16: 14.931591188925465
2020-08-17: 22.00130684683516
2020-08-18: 23.266488509340356
2

2020-08-06: 383.52294599951205
2020-08-07: 305.8422083570269
2020-08-08: 561.5144103879297
2020-08-09: 752.3108085884828
2020-08-10: 467.66651271917766
2020-08-11: 569.4850359736604
2020-08-12: 568.0581294119455
2020-08-13: 329.63183940000823
2020-08-14: 209.85548132098313
2020-08-15: 270.67550649548417
2020-08-16: 382.63844136247445
2020-08-17: 379.094223655243
2020-08-18: 362.9786730948902
2020-08-19: 287.7883823179316
2020-08-20: 170.0163793337941
2020-08-21: 142.32062603828516
2020-08-22: 218.87365965994476
2020-08-23: 182.5125308280645
2020-08-24: 225.94865386122544
2020-08-25: 191.19335639507162
2020-08-26: 160.92530352654433
2020-08-27: 101.36886019608329
2020-08-28: 75.78149003028251
2020-08-29: 110.15321348948801
2020-08-30: 141.23204371281042
2020-08-31: 121.77683113372153

Predicting for Belarus__nan
2020-08-01: 1071.0203494089676
2020-08-02: 1494.7427057151172
2020-08-03: 1371.378327197836
2020-08-04: 1405.1563644211933
2020-08-05: 957.0663645684873
2020-08-06: 711.06755434

2020-08-24: 208.02846679798282
2020-08-25: 77.76299377798222
2020-08-26: 74.92532832143486
2020-08-27: 129.50723858580008
2020-08-28: 169.05614645984832
2020-08-29: 197.6281821749524
2020-08-30: 151.71871745052363
2020-08-31: 115.07910245075367

Predicting for Brazil__Bahia
2020-08-01: 875.288647644146
2020-08-02: 2261.381795520622
2020-08-03: 1051.904021826379
2020-08-04: 424.6185309417788
2020-08-05: 485.13556771823505
2020-08-06: 760.6497194101062
2020-08-07: 1038.3565788755811
2020-08-08: 701.9253154455544
2020-08-09: 913.4690178993737
2020-08-10: 734.8984174592496
2020-08-11: 218.14989166384032
2020-08-12: 194.1550516029117
2020-08-13: 468.0938534666999
2020-08-14: 836.9346763441481
2020-08-15: 660.0519886036742
2020-08-16: 762.1357617482705
2020-08-17: 425.04390958328156
2020-08-18: 243.51114136047843
2020-08-19: 263.8455081444237
2020-08-20: 405.22240452464087
2020-08-21: 413.90208815097014
2020-08-22: 495.2916531104633
2020-08-23: 462.77295865278705
2020-08-24: 351.037747471193

2020-08-16: 585.4186229293991
2020-08-17: 422.3547869234839
2020-08-18: 287.0685248054351
2020-08-19: 264.7398392058578
2020-08-20: 355.35276640172464
2020-08-21: 387.55565630633583
2020-08-22: 477.34860059928326
2020-08-23: 549.2219438021303
2020-08-24: 364.5531182082545
2020-08-25: 143.1519959624485
2020-08-26: 150.64519809735057
2020-08-27: 261.4829112069053
2020-08-28: 194.66693751301892
2020-08-29: 275.61684976995633
2020-08-30: 318.90819723045496
2020-08-31: 207.339762361905

Predicting for Brazil__Paraiba
2020-08-01: 406.703376747169
2020-08-02: 680.4536171339752
2020-08-03: 752.8591447151464
2020-08-04: 407.3368632596182
2020-08-05: 192.0424209506318
2020-08-06: 407.9428003538984
2020-08-07: 524.7376982107495
2020-08-08: 604.9609686457278
2020-08-09: 640.6405185888532
2020-08-10: 481.4767857284608
2020-08-11: 167.04160513765706
2020-08-12: 171.419145089082
2020-08-13: 359.27718215252725
2020-08-14: 607.4394024377768
2020-08-15: 669.1157890499388
2020-08-16: 750.4375887420165
20

...

2020-08-29: 292.90670544097475
2020-08-30: 471.2146835101074
2020-08-31: 417.3065752250619

Predicting for United States__Massachusetts
2020-08-01: 3264.407071023588
2020-08-02: 4931.214535821417
2020-08-03: 4088.2784732874065
2020-08-04: 4683.415756995806
2020-08-05: 4405.196414641686
2020-08-06: 1513.6590885724818
2020-08-07: 2367.8764371702637
2020-08-08: 2766.9348637339654
2020-08-09: 2939.3826147065924
2020-08-10: 2029.944581370221
2020-08-11: 1849.507670508465
2020-08-12: 2500.1273261378274
2020-08-13: 1090.5409384268391
2020-08-14: 940.2634855403561
2020-08-15: 1129.7331286159965
2020-08-16: 1329.3307929361658
2020-08-17: 1178.3237202808702
2020-08-18: 1322.1313207030018
2020-08-19: 925.1979025535159
2020-08-20: 710.1966728892794
2020-08-21: 512.7372308118405
2020-08-22: 884.0425893444105
2020-08-23: 872.8606909491649
2020-08-24: 622.7152993511812
2020-08-25: 448.74155326955406
2020-08-26: 611.3574480990819
2020-08-27: 412.9409845476331
2020-08-28: 332.2888692350374
2020-08-29: 

2020-08-28: 412.9767846457668
2020-08-29: 356.7234999364315
2020-08-30: 482.21519182090645
2020-08-31: 369.3898814065573

Predicting for United States__North Dakota
2020-08-01: 197.37684108476012
2020-08-02: 497.7442489895719
2020-08-03: 338.95922441749303
2020-08-04: 301.75611102074026
2020-08-05: 180.67059397479258
2020-08-06: 183.89623746692126
2020-08-07: 158.73921599216447
2020-08-08: 159.57359373482024
2020-08-09: 314.36938609795027
2020-08-10: 218.2111044902669
2020-08-11: 245.4721148500097
2020-08-12: 162.89962117766677
2020-08-13: 97.61114002080238
2020-08-14: 96.93561698860252
2020-08-15: 88.7875513797289
2020-08-16: 215.08233192123555
2020-08-17: 156.6984713319669
2020-08-18: 160.72069051193546
2020-08-19: 83.44556667142888
2020-08-20: 58.879360086246294
2020-08-21: 63.760878226919424
2020-08-22: 52.39069444900398
2020-08-23: 118.32268126630915
2020-08-24: 101.28114006101869
2020-08-25: 86.933493852068
2020-08-26: 45.63442467244206
2020-08-27: 30.171061873252164
2020-08-28: 

2020-08-07: 850.6877512259443
2020-08-08: 1219.4611285257254
2020-08-09: 1104.5344491975159
2020-08-10: 807.8250739132234
2020-08-11: 687.6627584866634
2020-08-12: 1280.7192293838045
2020-08-13: 639.1583943238551
2020-08-14: 508.15003901624743
2020-08-15: 606.440682613244
2020-08-16: 712.1896950979655
2020-08-17: 459.974257106046
2020-08-18: 589.2063467953279
2020-08-19: 595.9334865482845
2020-08-20: 428.19278630021506
2020-08-21: 316.4644502068632
2020-08-22: 443.9884886420236
2020-08-23: 444.5252726746574
2020-08-24: 308.75171567333615
2020-08-25: 278.0504806108555
2020-08-26: 383.21258753847275
2020-08-27: 222.7553261457476
2020-08-28: 168.66694414820427
2020-08-29: 159.9366539826204
2020-08-30: 205.33414969979816
2020-08-31: 142.06351555341342

Predicting for United States__Oregon
2020-08-01: 817.605619589928
2020-08-02: 1280.227853753804
2020-08-03: 903.6020266891704
2020-08-04: 1098.6636239165437
2020-08-05: 753.0126083525666
2020-08-06: 549.3569267861764
2020-08-07: 632.71553738

2020-08-08: 1692.3965324718038
2020-08-09: 1178.0911433122528
2020-08-10: 1129.764272681906
2020-08-11: 1000.7042218475707
2020-08-12: 713.6211350165873
2020-08-13: 795.8220795120586
2020-08-14: 768.71890111319
2020-08-15: 835.2450871881908
2020-08-16: 739.0273166638731
2020-08-17: 623.8699723898335
2020-08-18: 770.7332867172382
2020-08-19: 663.1262656443012
2020-08-20: 574.8195752437673
2020-08-21: 426.29929967859056
2020-08-22: 544.6015649336185
2020-08-23: 501.78116759468776
2020-08-24: 379.9269209451597
2020-08-25: 289.2586610123682
2020-08-26: 410.84479724443247
2020-08-27: 320.86849713944144
2020-08-28: 236.07602075211588
2020-08-29: 175.8417612874261
2020-08-30: 286.64146636432724
2020-08-31: 198.36927476679057

Predicting for United States__Virgin Islands
2020-08-01: 14.225002053698843
2020-08-02: 10.635826083231514
2020-08-03: 11.156770813314447
2020-08-04: 9.968829372838185
2020-08-05: 11.217295800901432
2020-08-06: 14.113008100664956
2020-08-07: 11.681041095553514
2020-08-08

2020-08-01: 14.225002053698843
2020-08-02: 10.635826083231514
2020-08-03: 11.156770813314447
2020-08-04: 9.968829372838185
2020-08-05: 11.217295800901432
2020-08-06: 14.113008100664956
2020-08-07: 11.681041095553514
2020-08-08: 28.05107970852323
2020-08-09: 22.942656806282038
2020-08-10: 30.520995571047884
2020-08-11: 36.52274184160293
2020-08-12: 34.921356540236715
2020-08-13: 27.07261454704525
2020-08-14: 25.783807047184585
2020-08-15: 37.918467025113635
2020-08-16: 48.123298648394254
2020-08-17: 36.78244651708351
2020-08-18: 45.12070017073462
2020-08-19: 40.53896334275246
2020-08-20: 29.920424987323585
2020-08-21: 30.297649624345837
2020-08-22: 52.33087010751111
2020-08-23: 55.58487605071404
2020-08-24: 58.46457083663405
2020-08-25: 72.87469494330053
2020-08-26: 53.60824593720985
2020-08-27: 55.288149455615425
2020-08-28: 61.93025122071568
2020-08-29: 81.50750982661062
2020-08-30: 94.03535320352199
2020-08-31: 88.25478009965451

Predicting for Vietnam__nan
2020-08-01: 5.550147991387

In [43]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,11.290426
214,Aruba,NaN,2020-08-02,10.039194
215,Aruba,NaN,2020-08-03,11.425396
216,Aruba,NaN,2020-08-04,11.526868
217,Aruba,NaN,2020-08-05,8.034063


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [44]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [45]:
!head predictions/2020-08-01_2020-08-04.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,,2020-08-01,11.290425620760875
Aruba,,2020-08-02,10.039194205035018
Aruba,,2020-08-03,11.425396219616268
Aruba,,2020-08-04,11.526868044645925
Afghanistan,,2020-08-01,146.62497627378957
Afghanistan,,2020-08-02,89.12843873424981
Afghanistan,,2020-08-03,64.45813666067774
Afghanistan,,2020-08-04,64.9466436745464
Angola,,2020-08-01,87.5574555562355


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [46]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [47]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [48]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
CPU times: user 250 ms, sys: 160 ms, total: 410 ms
Wall time: 7.5 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [49]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-22
End date: 2021-06-20


In [50]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [51]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-22 to 2021-06-20...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
CPU times: user 7.3 s, sys: 1.03 s, total: 8.33 s
Wall time: 4min 13s
